# RF including Feature Engineering

In [91]:
import pandas as pd
import numpy as np
from helper import * 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


In [92]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

# Import data and clean with wrapper

In [93]:
# importing data
housing = pd.read_csv('Ames_Housing_Price_Data.csv', index_col=0,low_memory = False)
# data processing
train, test = data_processing_wrapper(housing,
                                               num_to_cat_list = ['MSSubClass','MoSold'],
                                             remove_PID = False
                                        )

# feature engineering wrapper
train, test = feature_engineering_wrapper(train, test)

# importing school feature
schools = pd.read_csv('schoolFeatures.csv',index_col = 0)
school_keep = [
    'PID',
    'closestSchool'
]
schools = schools[school_keep]

# merge school feature onto original data set.
train = train.merge(schools, how = 'left', left_on = 'PID', right_on = 'PID')
test = test.merge(schools, how = 'left', left_on = 'PID', right_on = 'PID')

train = train.dropna(subset=['closestSchool'])
train = train.reset_index(drop=True)

test = test.dropna(subset=['closestSchool'])
test = test.reset_index(drop=True)

In [94]:
#housing started out with 81 columns (2580 rows)
#feature engineering - ended up with 129 columns (including PID and SalePrice)-- (added 48 new columns) (2477 rows between train and test)

# Split into predictors and target

In [95]:
train_raw = train.copy()
test_raw = test.copy()

train_X = train_raw.drop(['SalePrice','PID'],axis='columns')
train_y = train_raw['SalePrice']
test_X = test_raw.drop(['SalePrice','PID'],axis='columns')
test_y = test_raw['SalePrice']

In [96]:
train_y_log = np.log(train_y).rename('LogSalePrice')
test_y_log = np.log(test_y).rename('LogSalePrice')

In [97]:
cat_feats = train_X.select_dtypes(['object']).columns.to_list()
num_feats = train_X.select_dtypes(['int','float']).columns.to_list()

In [98]:
len(cat_feats) # 32 categorical features (43 originally in housing dataset)
len(num_feats) #95 numeric features (38 originally)

95

# Dummify 

In [99]:
#Preprocessing / Dummification
preprocessor = ColumnTransformer(transformers=[
    ('tf1',OneHotEncoder(sparse=False, handle_unknown='ignore'), cat_feats)],remainder='passthrough')

train_X_transformed = preprocessor.fit_transform(train_X)

In [100]:
#Get one-hot encoded column names 
columns_transformed = preprocessor.named_transformers_['tf1'].get_feature_names(input_features= cat_feats)
new_columns = list(columns_transformed)+num_feats

#Place one-hot encoded train X into dataframe 
train_X_transformed = pd.DataFrame(train_X_transformed,columns=new_columns)

#Repeat for test X 
test_X_transformed = preprocessor.transform(test_X)
test_X_transformed = pd.DataFrame(test_X_transformed,columns=new_columns)

# Filter down Features Based off of LASSO 

In [101]:
#Features selected by LASSO model (non-zero coefficients)
coef_df = pd.read_csv('lasso_coef.csv',index_col=0) #Hayden shared this on Slack

In [102]:
selected_features = list(coef_df['features'])

In [103]:
print('Number of LASSO selected features: '+ str(len(selected_features)))

Number of LASSO selected features: 140


In [104]:
print('Number of total features after preprocessing: ' + str(len(train_X_transformed.columns.to_list())))

Number of total features after preprocessing: 323


In [105]:
#Filter dataframes down to the select_features 
train_X= train_X_transformed[selected_features]
test_X = test_X_transformed[selected_features]

# Random Forest Modeling 

In [106]:
#Function to print scores
def rf_summarize(model,Xtrain,ytrain,Xtest,ytest,num=10):

    print(model.get_params())
    print('\n')
    print('Cross Val score: ', cross_val_score(model, Xtrain, ytrain, cv=5))
    print('Cross Val score mean: ', cross_val_score(model, Xtrain, ytrain, cv=5).mean())
    print('Train score: ',model.score(Xtrain,ytrain))
    print('Test score: ',model.score(Xtest,ytest))
    print('\n')
    importances = pd.Series(model.feature_importances_, index = Xtrain.columns).sort_values(ascending=False)
    print(importances[:num])

## 1. No Tuning

### 1A. Feature Selection

In [107]:
rf = RandomForestRegressor(random_state=0)
rf.fit(train_X, train_y_log)
rf_summarize(rf,train_X,train_y_log,test_X,test_y_log)

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


Cross Val score:  [0.92436102 0.93526875 0.92429566 0.92783612 0.91877842]
Cross Val score mean:  0.9261079946056376
Train score:  0.98945671606562
Test score:  0.8984127748003494


OverallQual_cubed               0.428616
total_sf_square_root            0.289976
total_high_qual_finished_sf     0.091691
Neighborhood_log_comp           0.043746
overall_score_square_root       0.035583
GarageArea                      0.012643
year_since_built_square_root    0.011987
GrLivArea_square_root           0.009343
YearRemodAdd                    0.006577
LotArea_squared                 0.005978
dtype: floa

In [109]:
#Try with SalePrice instead of log
rf = RandomForestRegressor(random_state=0)
rf.fit(train_X, train_y)
rf_summarize(rf,train_X,train_y,test_X,test_y)

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


Cross Val score:  [0.94446388 0.91958566 0.91982172 0.93378899 0.91650559]
Cross Val score mean:  0.9268331674017437
Train score:  0.9892740184526548
Test score:  0.8661877287542651


OverallQual_cubed               0.399807
total_sf_square_root            0.350979
total_high_qual_finished_sf     0.077981
Neighborhood_log_comp           0.055797
overall_score_square_root       0.017134
year_since_built_square_root    0.009403
GrLivArea_square_root           0.008955
GarageArea                      0.008135
YearRemodAdd                    0.007042
LotArea_square_root             0.005311
dtype: fl

### 1B. All Features

In [ ]:
#No feature selection

In [110]:
rf = RandomForestRegressor(random_state=0)
rf.fit(train_X_transformed, train_y_log)
rf_summarize(rf,train_X_transformed,train_y_log,test_X_transformed,test_y_log)

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


Cross Val score:  [0.92248157 0.93275171 0.92364538 0.9245361  0.91484611]
Cross Val score mean:  0.9236521747474592
Train score:  0.9891372042859093
Test score:  0.8952997288692605


OverallQual                    0.118576
OverallQual_square_root        0.104625
OverallQual_cubed              0.103060
OverallQual_squared            0.101697
total_high_qual_finished_sf    0.088775
total_sf                       0.081656
total_sf_square_root           0.073389
total_sf_cubed                 0.071333
total_sf_squared               0.060984
Neighborhood_log_comp          0.042242
dtype: float64


In [111]:
#remove log on target 
rf = RandomForestRegressor(random_state=0)
rf.fit(train_X_transformed, train_y)
rf_summarize(rf,train_X_transformed,train_y,test_X_transformed,test_y)

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


Cross Val score:  [0.94421031 0.91529305 0.91735592 0.92721745 0.91035143]
Cross Val score mean:  0.9228856324406379
Train score:  0.9888767085758192
Test score:  0.8638453763909857


OverallQual_squared            0.123926
total_sf_cubed                 0.105298
OverallQual                    0.104546
total_sf_square_root           0.104352
OverallQual_cubed              0.087981
OverallQual_square_root        0.082700
total_high_qual_finished_sf    0.074941
total_sf_squared               0.070564
total_sf                       0.068816
Neighborhood_log_comp          0.053998
dtype: float64


## 2. Tuning

### 2A. Feature Selection

In [ ]:
# # Number of trees in random forest
# n_estimators = [100,200,400,500,600]
# # Criterion to measure quality of split
# criterion = ["mse", "mae"]
# # Number of features to consider at every split
# max_features = [round(i,2) for i in list(np.arange(0.1,1,.1))] + ['auto','sqrt']
# # Maximum number of levels in tree
# max_depth = list(range(10,111,10)) + ['None']
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 10]
# # Method of selecting samples for training each tree
# bootstrap = [True]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'criterion': criterion,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

In [ ]:
#random_grid

In [ ]:
# {'n_estimators': [100, 200, 400, 500, 600],
#  'criterion': ['mse', 'mae'],
#  'max_features': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 'auto', 'sqrt'],
#  'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 'None'],
#  'min_samples_split': [2, 5, 10],
#  'min_samples_leaf': [1, 10],
#  'bootstrap': [True]}


# {'n_estimators': [100, 400],
#  'criterion': ['mse'],
#  'max_features': [0.3, 0.35, 0.4, 'auto', 'sqrt'],
#  'max_depth': [30, 31, 32, 33, 34, 35, 40, 'None'],
#  'min_samples_split': [2],
#  'min_samples_leaf': [1],
#  'bootstrap': [True]}

# random_grid3 = {'n_estimators': [400,450,500],
#  'min_samples_split': [2],
#  'min_samples_leaf': [1],
#  'max_features': [0.25,0.26,0.27,0.28,0.29,0.3],
#  'max_depth': [34],
#  'criterion': ['mse'],
#  'bootstrap': [True]}

In [ ]:
# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, 
#                                param_distributions = random_grid, 
#                                n_iter = 50, 
#                                cv = 5,
#                                verbose=2, 
#                                random_state=42, 
#                                n_jobs = -1)
# # Fit the random search model
# rf_random.fit(train_X, train_y_log)

In [ ]:
#rf_random.best_params_

In [ ]:
# Followup with grid search 
#grid_params = {'n_estimators': [475],
#  'max_features': [0.28],
#  'max_depth': [15,16,17,18,19,20,21,22,23,24,25]}

# rf = RandomForestRegressor()
# rf_gridsearch = GridSearchCV(estimator = rf, 
#                    param_grid = grid_params, 
#                    cv = 5,
#                    verbose=2, 
#                    n_jobs = -1,
#                    return_train_score=True)
# rf_gridsearch.fit(train_X, train_y_log)


In [ ]:
# {'n_estimators': 100,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 0.3,
#  'max_depth': 30,
#  'bootstrap': True}
# Cross Val score:  [0.93579475 0.92828102 0.92433193]
# Cross Val score mean:  0.9282121647223146
# Train score:  0.9905555677143995
# Test score:  0.9104753934134463

# {'n_estimators': 400,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 0.4,
#  'max_depth': 40,
#  'criterion': 'mse',
#  'bootstrap': True}
# Cross Val score:  [0.93583252 0.92783454 0.92602133]
# Cross Val score mean:  0.9288421660609751
# Train score:  0.9911133519550306
# Test score:  0.9097288447153569


# {'n_estimators': 400,
#  'min_samples_split': 5,
#  'min_samples_leaf': 1,
#  'max_features': 0.4,
#  'max_depth': 30,
#  'criterion': 'mse',
#  'bootstrap': True}
# Cross Val score:  [0.93560239 0.92652314 0.92355869]
# Cross Val score mean:  0.9294543039840469
# Train score:  0.9873146849429972
# Test score:  0.9099829502773876

# {'n_estimators': 400,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 0.3,
#  'max_depth': 34,
#  'criterion': 'mse',
#  'bootstrap': True}

# Cross Val score:  [0.93631747 0.92722897 0.92696265]
# Cross Val score mean:  0.9293809452490164
# Train score:  0.9908571241679297
# Test score:  0.9114062426467685

# {'n_estimators': 450,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 0.27,
#  'max_depth': 34,
#  'criterion': 'mse',
#  'bootstrap': True}

# Cross Val score:  [0.93587818 0.92713536 0.92576214]
# Cross Val score mean:  0.9291140950912293
# Train score:  0.9908353458911427
# Test score:  0.9110840400893874

# {'n_estimators': 450,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 0.27,
#  'max_depth': 34,
#  'criterion': 'mse',
#  'bootstrap': True}

#Grid search
#{'max_depth': 23, 'max_features': 0.28, 'n_estimators': 475}
# Cross Val score:  [0.93706284 0.92701748 0.92542083]
# Cross Val score mean:  0.929663346136536
# Train score:  0.9909447830148265
# Test score:  0.920584632903027

#{'max_depth': 21, 'max_features': 0.28, 'n_estimators': 475}
# Cross Val score:  [0.93550739 0.92688435 0.92573597]
# Cross Val score mean:  0.9296219950550052
# Train score:  0.9908067817643416
# Test score:  0.9113437650102342


In [146]:
random_forest = RandomForestRegressor(n_estimators=475, 
                                      max_depth=23, 
                                      max_features=0.28, 
                                      n_jobs=-1,
                                      random_state=0,
                                      )

In [149]:
random_forest.fit(train_X,train_y_log)

RandomForestRegressor(max_depth=23, max_features=0.28, n_estimators=475,
                      n_jobs=-1, random_state=0)

In [150]:
rf_summarize(random_forest,train_X, train_y_log, test_X, test_y_log,25)

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': 23, 'max_features': 0.28, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 475, 'n_jobs': -1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


Cross Val score:  [0.93289584 0.93699087 0.92947119 0.92741967 0.92810134]
Cross Val score mean:  0.9309757800768788
Train score:  0.9909824689781674
Test score:  0.9109286902705149


total_sf_square_root            0.219555
OverallQual_cubed               0.191229
total_high_qual_finished_sf     0.115746
Neighborhood_log_comp           0.091265
GrLivArea_square_root           0.054094
year_since_built_square_root    0.045144
ExterQual                       0.038058
overall_score_square_root       0.034254
GarageArea                      0.025890
GarageCars_log_comp             0.023409
GarageCars     

In [115]:
#Results for tuned rf with feature selection
# {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': 23, 'max_features': 0.28, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 475, 'n_jobs': -1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


# Cross Val score:  [0.93289584 0.93699087 0.92947119 0.92741967 0.92810134]
# Cross Val score mean:  0.9309757800768785
# Train score:  0.9909824689781674
# Test score:  0.9109286902705149


# total_sf_square_root            0.219555
# OverallQual_cubed               0.191229
# total_high_qual_finished_sf     0.115746
# Neighborhood_log_comp           0.091265
# GrLivArea_square_root           0.054094
# year_since_built_square_root    0.045144
# ExterQual                       0.038058
# overall_score_square_root       0.034254
# GarageArea                      0.025890
# GarageCars_log_comp             0.023409
# GarageCars                      0.016580
# BsmtQual                        0.015127
# KitchenQual_cubed               0.014063
# LotArea_squared                 0.009416
# LotArea_square_root             0.009276
# YearRemodAdd                    0.008412
# BsmtFinSF1                      0.007112
# MasVnrArea                      0.005505
# OverallCond                     0.005320
# FireplaceQu                     0.005196
# total_deck_sf                   0.004939
# garage_score                    0.004545
# Fireplaces                      0.004404
# TotRmsAbvGrd                    0.003296
# total_full_bath                 0.003209

### 2B. All Features

In [ ]:
# {'n_estimators': 400,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 0.1,
#  'max_depth': 40,
#  'criterion': 'mse',
#  'bootstrap': True}

# Cross Val score:  [0.93226744 0.93403861 0.922266   0.92518766 0.92206218]
# Cross Val score mean:  0.9269950077031422
# Train score:  0.9903395273834038
# Test score:  0.9064977926272272


# {'n_estimators': 200,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 0.1,
#  'max_depth': 20,
#  'criterion': 'mse',
#  'bootstrap': True}

# Cross Val score:  [0.93378291 0.93296664 0.92223692 0.92660782 0.92151614]
# Cross Val score mean:  0.9263690029773128
# Train score:  0.9903314771724289
# Test score:  0.9031112285209117

#grid search
#{'max_depth': 36, 'max_features': 0.1, 'n_estimators': 300}


# Cross Val score:  [0.93243304 0.93325527 0.92323578 0.92613059 0.92045473]
# Cross Val score mean:  0.9267487110711168
# Train score:  0.9901995039746295
# Test score:  0.9049766012854378


In [118]:
# random_grid = {'n_estimators': [100, 200, 400,425,450,475,500],
#  'criterion': ['mse'],
#  'max_features': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 'auto', 'sqrt'],
#  'max_depth': [10, 20, 30,35, 40, 50, 60, 'None'],
#  'min_samples_split': [2],
#  'min_samples_leaf': [1],
#  'bootstrap': [True]}

In [142]:
# # Use the random grid to search for best hyperparameters
# # First create the base model to tune
# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, 
#                                param_distributions = random_grid, 
#                                n_iter = 50, 
#                                cv = 5,
#                                verbose=2, 
#                                random_state=42, 
#                                n_jobs = -1)
# # Fit the random search model
# rf_random.fit(train_X_transformed, train_y_log)

In [143]:
# #Followup with grid search 
# grid_params = {'n_estimators': [200,250,300,350,400,450,475],
#  'max_features': [0.1],
#  'max_depth': list(range(20,45,1))}

# rf = RandomForestRegressor()
# rf_gridsearch = GridSearchCV(estimator = rf, 
#                    param_grid = grid_params, 
#                    cv = 5,
#                    verbose=2, 
#                    n_jobs = -1,
#                    return_train_score=True)
# rf_gridsearch.fit(train_X_transformed, train_y_log)

In [139]:
random_forest = RandomForestRegressor(n_estimators=300, 
                                      max_depth=36, 
                                      max_features=0.1, 
                                      n_jobs=-1,
                                      random_state=0,
                                      )

In [145]:
random_forest.fit(train_X_transformed,train_y_log)
rf_summarize(random_forest,train_X_transformed,train_y_log,test_X_transformed,test_y_log,25)

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': 36, 'max_features': 0.1, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


Cross Val score:  [0.93273479 0.93491204 0.92292383 0.92370534 0.92157475]
Cross Val score mean:  0.9271701501791497
Train score:  0.990276972313275
Test score:  0.9042749333234227


total_sf_squared               0.069766
total_sf_cubed                 0.065509
total_sf                       0.065176
OverallQual                    0.064999
OverallQual_squared            0.062314
total_sf_square_root           0.056356
OverallQual_cubed              0.052879
OverallQual_square_root        0.052504
total_high_qual_finished_sf    0.041294
Neighborhood_log_comp          0.035703
GrLivArea_squared          

In [ ]:
# #Results tuned RF with all features 
# {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': 36, 'max_features': 0.1, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


# Cross Val score:  [0.93273479 0.93491204 0.92292383 0.92370534 0.92157475]
# Cross Val score mean:  0.9271701501791497
# Train score:  0.990276972313275
# Test score:  0.9042749333234227


# total_sf_squared               0.069766
# total_sf_cubed                 0.065509
# total_sf                       0.065176
# OverallQual                    0.064999
# OverallQual_squared            0.062314
# total_sf_square_root           0.056356
# OverallQual_cubed              0.052879
# OverallQual_square_root        0.052504
# total_high_qual_finished_sf    0.041294
# Neighborhood_log_comp          0.035703
# GrLivArea_squared              0.021725
# GrLivArea_square_root          0.018254
# YearBuilt                      0.018155
# GrLivArea                      0.016954
# ExterQual                      0.016779
# GrLivArea_cubed                0.016003
# year_since_built               0.013609
# GarageArea                     0.012863
# overall_score_square_root      0.012749
# year_since_built_cubed         0.012197
# GarageArea_cubed               0.011254
# overall_score_squared          0.011082
# overall_score_cubed            0.010948
# GarageCars_log_comp            0.010864
# GarageArea_squared             0.009674
# dtype: float64